# Programação orientada a objetos

Este notebook é a segunda parte de uma adaptação do tutorial ["Python 3 Tutorial"](https://www.python-course.eu/python3_object_oriented_programming.php), do site [python-course.eu](https://www.python-course.eu).

## Tudo de primeira classe

Ainda que a gente não tenha falado sobre classes e orientação a objetos até aqui, nós trabalhamos com classes o tempo todo. De fato, tudo em Python é um objeto.

Guido van Rossum projetou a linguagem seguindo o princípio "*tudo de primeira classe*" ("*first-class everything*") ([neste post](http://python-history.blogspot.com/2009/02/first-class-everything.html) ele fala sobre isso, mas o foco era Python 2). 

Uma das muitas classes embutidas em Python é a classe `list`, que temos usado bastante em nossos exemplos. No exemplo abaixo, as referências `x` e `y` são associadas a duas instâncias da classe `list`. Simplificando, dissemos que `x` e `y` são listas. A partir de agora, vamos usar os termos **instância** e **objeto** como sinônimos.

In [1]:
x = [3,6,9]
y = [45, "abc"]
print(x[1])

x[1] = 99
x.append(42)
last = y.pop()
print(last)

6
abc


Note que nós conseguimos acessar e manipular dados contidos em uma lista através de seus métodos, mesmo sem saber como estes métodos estão representados internamente. 

Vamos discutir um pouco mais sobre isso.

## Abstração de dados, encapsulamento de dados e ocultação de informação

Esses três termos costumam ser usados como sinônimos em livros e tutoriais sobre POO, mas há uma diferença entre eles.

### Encapsulamento de dados
Encapsulamento é a capacidade de ter atributos e métodos juntos em uma mesma estrutura:

In [2]:
class Person:
    def __init__(self, name):
        self.name = name

    def __str__(self):
        return self.name
        
    def change_name(self, name):
        self.name = name

In [3]:
joao = Person("Joao")
maria = Person("Maria")
print(joao)
print(maria)

Joao
Maria


In [4]:
joao.change_name("João")
print(joao)

João


In [5]:
print(joao.name)
joao.name = "Jorge"
print(joao)

João
Jorge


In [6]:
joao.sexo = "M"
print(joao)

Jorge


### Ocultação de informação

Por outro lado, ocultação de informação é a proteção de dados para evitar sua alteração. Em Python, isto pode ser feito acrescentando um `__` ao início do nome do atributo que se deseja ocultar:

In [7]:
class Person:
    def __init__(self, name, age):
        self.name = name
        self.__age = age

    def __str__(self):
        return "{}: {}".format(self.name, self.__age)
        
    def change_name(self, name):
        self.name = name

In [8]:
joao = Person("Joao", 23)
maria = Person("Maria", 33)
print(joao)
print(maria)

Joao: 23
Maria: 33


In [9]:
joao.change_name("João")
print(joao)
maria.age = 25
print(maria)

João: 23
Maria: 33


É importante ressaltar que um atributo que tenha início com o prefixo `__` não é ocultado completamente do mundo externo à classe em que é definido.

O que o interpretador Python faz é renomear esses atributos para o padrão `_Classe__atributo`:

In [10]:
print(maria._Person__age)
maria._Person__age = 28
print(maria._Person__age)

33
28


Esse procedimento é conhecido como *desfiguração de nomes*. 

**O objetivo desse procedimento é evitar que um atributo seja alterado acidentalmente - não intencionalmente!**

Também é possível usar apenas um underscore antes de um nome para dizer que um atributo não deve ser acessado fora de uma classe -- mas isto é apenas um acordo entre programadores (uma convenção da comunidade) e não um recurso da linguagem.

### Abstração de dados

O encapsulamento de dados através de métodos não significa necessariamente que a informação esteja oculta. A simples existência de métodos para acessos a atributos não restringe o acesso direto a eles, mesmo que seja recomendável fazer uso dos métodos:

In [11]:
joao.name = "José"
print(joao)

José: 23


Assim, a abstração de dados é obtida se tanto o encapsulamento de dados como a ocultação da informação forem adotadas:

**Abstração de dados = Encapsulamento de dados + Ocultação de informação**

Note que para isto acontecer, dados que são considerados ocultos não devem ser acessados externamente. Na prática, no entanto, é comum que dados ocultos sejam acessados através de acessores (*getters*) e modificadores (*setters*). É comum que IDEs até reclamem se você esquecer de implementar getters e setters para um atributo privado!

A forma pythônica de lidar com abstração de dados é evitar a ocultação de informação, adotando como convenção que todo atributo seja público.

No entanto, é possível que ao longo do ciclo de vida do seu código um atributo precise ser ocultado. Em Python, usamos propriedades para controlar o acesso/modificação de um atributo:

In [12]:
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age

    def __str__(self):
        return "{}: {}".format(self.name, self.age)

    @property
    def name(self):
        print("Getting name")
        return self.__name
    
    @name.setter
    def name(self, name):
        print("Setting name")
        self.__name = name
        
    @property
    def age(self):
        print("Getting age")
        return self.__age
    
    @age.setter
    def age(self, age):
        print("Setting age")
        if age < 0:
            raise ValueError
        else:
            self.__age = age

In [13]:
Person("Leo",-2)

Setting name
Setting age


ValueError: 

In [14]:
print("-- creating joao --")
joao = Person("Joao", 23)
print("-- creating maria --")
maria = Person("Maria", 33)
print("-- printing joao --")
print(joao)
print("-- printing maria --")
print(maria)

-- creating joao --
Setting name
Setting age
-- creating maria --
Setting name
Setting age
-- printing joao --
Getting name
Getting age
Joao: 23
-- printing maria --
Getting name
Getting age
Maria: 33


In [15]:
print("-- changing joao --")
joao.name = "João"
print("-- printing joao --")
print(joao)
print("-- changing maria --")
maria.age = -1
print("-- printing maria --")
print(maria)

-- changing joao --
Setting name
-- printing joao --
Getting name
Getting age
João: 23
-- changing maria --
Setting age


ValueError: 

In [16]:
maria._Person__age = -1

In [17]:
print(maria)

Getting name
Getting age
Maria: -1


Note que tanto internamente como externamente a classe Person trata os atributos `name` e `age` como se fossem públicos.

**O intuito do decorador `@property` é permitir o tratamento do acesso e da modificação dos atributos presentes em uma classe, não evitá-lo!**

Há duas vantagens principais na abordagem adotada pelo Python:
- a legibilidade do código sem getters e setters é consideravelmente maior. Compare a versão com `@property`

In [18]:
print("-- changing joao --")
joao.name += " Maria"
print("-- printing joao --")
print(joao)

-- changing joao --
Getting name
Setting name
-- printing joao --
Getting name
Getting age
João Maria: 23


com a versão sem `@property`:

`joao.set_name(joao.get_name() + " Maria")`

- seu código pode começar de forma simples, sendo incrementado apenas quando necessário. Veja que o exemplo abaixo é retrocompatível com a classe Person definida anteriormente:

In [19]:
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age

    def __str__(self):
        return "{}: {}".format(self.name, self.age)

joao = Person("Joao", 23)
maria = Person("Maria", 33)
print(joao)
print(maria)

joao.name = "João"
print(joao)
maria.age = 25
print(maria)

Joao: 23
Maria: 33
João: 23
Maria: 25


Assim, você pode escolher adicionar ou remover acessores e modificadores ao longo da existência do seu código sem que o usuário note a diferença ;)

### Outros usos do decorador `@property`

Usando `@property` é possível criar um acessor sem um modificador, mas não o contrário. Quando fazemos isto, estamos dizendo que o atributo é somente leitura:

In [20]:
class Person:
    def __init__(self, name, age):
        self.__name = name
        self.age = age

    def __str__(self):
        return "{}: {}".format(self.name, self.age)
    
    @property
    def name(self):
        return self.__name

joao = Person("Joao", 23)
maria = Person("Maria", 33)
print(joao)
print(maria)

joao.name = "João"
print(joao)
maria.age = 25
print(maria)

Joao: 23
Maria: 33


AttributeError: can't set attribute

Também é possível definir uma `@property` a partir de um método:

In [21]:
class Person:
    def __init__(self, name, last_name, age):
        self.name = name
        self.last_name = last_name
        self.age = age

    def __str__(self):
        return "{}: {}".format(self.fullname, self.age)
    
    @property
    def fullname(self):
        return "{} {}".format(self.name, self.last_name)

joao = Person("Joao", "Silva", 23)
maria = Person("Maria", "Santos", 33)
print(joao)
print(maria)

joao.name = "João"
print(joao)
maria.age = 25
print(maria)

Joao Silva: 23
Maria Santos: 33
João Silva: 23
Maria Santos: 25


Note que a `@property` criada fica visível também fora da classe:

In [22]:
print(joao.fullname)

João Silva


Mas a property não é considerada um atributo do objeto:

In [23]:
print(joao.__dict__)

{'name': 'João', 'last_name': 'Silva', 'age': 23}


**Obs.: não faz sentido criar um modificador para esta `@property`.**